In [4]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from chat_memory import get_chat_history, last_n_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
import textwrap

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma(persist_directory="path/to/your/chroma_db", embedding_function=embeddings)

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Router

In [5]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Your job is to classify user queries into two buckets: 
     - memory
     - memory and vectorstore
     
     *Guidelines*
     Questions regarding exercises, exercise form, or technique: vectorstore & memory
     General question or statements: memory
   
     *Respond to queries with only the correct class*
     Examples: 
     User: how should I grip the bar for bench press?
     Agent: vectorstore & memory

     User: what muscles should I feel during the Romanian dead lift?
     Agent: vectorstore & memory

     User: How's this? 
     Agent: vectorstore & memory

     User: Is this good squat technique? 
     Agent: vectorstore & memory

     User: Was my bar path better?
     Agent: vectorstore & memory

     User: thakns for your help!
     Agent: memory

"""),

     ("human", "{query}")
     
])

llm = ChatOpenAI(model='gpt-4o')

output_parser = StrOutputParser()

router_chain = router_prompt | llm | output_parser

router_response = router_chain.invoke({"query": "hi"})

print(router_response)

memory


In [6]:
def router(user_query):
    route = router_chain.invoke({"query": user_query})
    route = route.lower().strip()
    return route


router("thanks for everything!")

'memory'

# Orchestrator

In [7]:
 
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a world-class fitness coach. You have extensive experience in helping weight lifters achieve perfect form and maximum hypertrophy. 
Your job is to analyze images of users lifting weights, offer them advice from your context, and to answer any questions they might have. 
Inspect each image CLOSELY and arefuly for problems or issues related to best practices in exercise form. Help the user diagnose their incorrect form. 
Be specific about what you observe.

# ANSWER CONTEXT
Use ONLY the following context when answering a user: 
     
---   
{context}
 ---
"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{query}")
])

llm = ChatOpenAI(model='gpt-5',
                 temperature=0.5)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"

)

response = chain_with_history.invoke(
    {"query": "what's good grip for bench press?", "context": ""},
    config={"configurable": {"session_id": "test_123"}}
)

print(textwrap.fill(response))

print(response)

Great question. “Good” depends on your build and goal, but here’s a
quick way to dial in a safe, strong bench grip:  How wide - Start with
a moderate width: about 1.4–1.8x your shoulder (acromion-to-acromion)
width. For most, that’s ring or middle finger on the outer rings of a
power bar (rings are 81 cm apart). - Quick check: at the bottom (bar
on lower sternum), your forearms should be vertical from the front
view. If your forearms angle in, go wider; if they angle out, go
narrower. - Goal tweaks:   - Max strength/pec focus: slightly wider
(closer to the rings) if shoulders tolerate it.   - Triceps/shoulder-
friendly: shoulder-width to just outside (1.0–1.3x), a “close” but not
narrow grip.  Hand and wrist setup - Use a full grip (thumb wrapped).
Avoid thumbless “suicide” grip. - Place the bar low in your palm over
the heel of the hand, not in the fingers. Stack bar → wrist → elbow in
one line. - Keep wrists only slightly extended; if they’re bent back a
lot, move the bar lower in yo

In [11]:
def handle_query(user_query, session_id): # create function for handling queries 
    route = router(user_query) # pass route variable into function ith router function
    print(route)

    if "vectorstore" in route: # if route contains vectorstore 
        results = vectorstore.similarity_search(user_query, k=4) # pass the user query to similariy search (vectorstore)
        context = "\n".join([r.page_content for r in results]) # pass each of the results (k=n), joined, into the context to be read by image analyzer
    else: 
        context = "" # context is blank but we stil pass the query


    # now call the LLM with memory
    response = chain_with_history.invoke( 
        {"query": user_query, "context": context}, # pass the user query and the context to the chat_memory (vectorstore & memory)
        config={"configurable": {"session_id": session_id}} # initalize or call the chat's session_id
    )
    return response # return a resopnse that uses vectorstore & memory

In [9]:
handle_query("what's a good grip for bench press?", "test_456")

'Great question. A “good grip” on bench press keeps your wrists, elbows, and shoulders stacked so you can press safely and strongly.\n\nWhat to aim for\n- Width: Choose a width that makes your forearms vertical when the bar touches mid-to-lower sternum.\n  - Quick start: Place ring finger to index finger on the bar’s power rings (most people land between pinky-on-rings and index-on-rings). Adjust to get vertical forearms at the bottom.\n- Wrist position: Bar sits low in the palm (near the wrist crease), not in the fingers. Knuckles to the ceiling. Wrist stacked directly over the elbow.\n- Thumb: Use a full, closed grip (wrap the thumb) for safety and better force transfer.\n- Elbows: Tuck about 30–45° from your torso as you lower; don’t flare straight out.\n- Bar path: Down to mid/lower sternum, up over the shoulder joint.\n\nHow to find your exact width (2-minute setup)\n1) Unrack an empty bar. Set your shoulder blades “down and back.”\n2) Lower to mid/lower sternum and pause.\n3) Loo

In [10]:
handle_query("thanks for the help!", "test_456")

'You’re welcome! If you want, send a 10–15s front and side clip at your working weight and I’ll fine-tune hand width, wrist stack, elbow angle, and bar path. I can also suggest shoulder-friendly variations and a warm-up if you need it.'

In [12]:
handle_query("sounds good!", "test_456")

memory


'Perfect. Send two short clips at your working weight and I’ll dial everything in.\n\nHow to film (10–15s each)\n- Front angle: 30–45° off center, camera at mid‑chest height, 6–10 ft away. Include the whole bar, your torso, and elbows.\n- Side angle: Perpendicular to the bench, camera at mid‑chest height. Capture feet-to-bar so I can see leg drive, arch, butt/shoulder contact, bar path.\n\nWhat to show\n- 3–5 reps at normal tempo, include the unrack and re‑rack.\n- A 2–3s close‑up of your hand placement on the knurling before the set.\n- Keep plates and collars visible so I can confirm load.\n\nWhat to tell me\n- Exact grip (which finger on the power rings), bar weight, and whether you pause or touch‑and‑go.\n- Any discomfort (wrists, elbows, shoulders) and your main goal (strength, hypertrophy, shoulder‑friendly, etc.).\n\nSafety\n- Use a spotter or set safeties just below chest height when the bar is on you.\n- Use collars.\n\nIf you can’t film, I can still help—tell me your shoulder

In [13]:
handle_query("what about my foot placement?", "test_456")

vectorstore & memory


'Great question—foot placement makes or breaks your leg drive and upper‑back stability on bench.\n\nWhat to aim for\n- Stance width: About shoulder-width to slightly wider.\n- Toes: Turned out 15–30° so you can “screw” feet into the floor.\n- Position: Feet slightly behind your knees. Shins vertical to slightly angled back at the bottom. This lets you drive back toward your head without your butt popping up.\n- Contact: Full “tripod” on each foot—big toe, little toe, and heel glued to the floor the entire set.\n- Direction of force: Push the floor away/back toward your head to wedge your shoulders harder into the bench, not straight up.\n\nSimple setup (30 seconds)\n1) Set your shoulder blades down and back.\n2) Plant feet where you can keep full tripod contact; angle them slightly behind your knees.\n3) Pre‑load leg drive by gently pushing back toward your head until you feel quad tension and your upper back locks in.\n4) Unrack without losing that pressure. Keep steady leg drive thro